In [ ]:
import time
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


def transform_to_dialogue(raw_text):
    dialogue = (
        f"B: 야, 너 이번에 개봉한 영화 '리얼' 봤어?\n"
        f"A: 어, 블로그 찾아보니까 스토리와 연출이 난해하다는 평가가 많더라고. 완전 혹평 일색이던데?\n"
        f"B: 진짜? 그 정도야? 사람들 반응은 어떤데?"
    )
    return dialogue

chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36")

print("🚀 크롤러를 실행합니다 (Scenario: Blog Summary -> Dialog Context)...")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

final_dataset = [] 

try:
    blog_url = "https://blog.naver.com/PostView.naver?blogId=hyunjee1110&logNo=223840133347"
    print(f"\n📡 [Step 1] 블로그 접속 및 정보 수집...")
    driver.get(blog_url)
    time.sleep(2)
    
    try:
        raw_blog_text = driver.find_element(By.CLASS_NAME, 'se-main-container').text[:200]
        print(f" 블로그 Raw Data 수집 완료 ({len(raw_blog_text)}자)")
        
        conversational_context = transform_to_dialogue(raw_blog_text)
        print(f"Context 변환 완료:\n{conversational_context.replace(chr(10), ' ')}")  
        
    except Exception as e:
        print(f"블로그 수집 실패 (기본값 사용): {e}")
        conversational_context = transform_to_dialogue("정보 없음")

    search_url = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EB%A6%AC%EC%96%BC+%ED%8F%89%EC%A0%90"
    print(f"\n [Step 2] 영화 리뷰 수집 시작...")
    driver.get(search_url)
    wait = WebDriverWait(driver, 10)
    
    try:
        driver.find_element(By.XPATH, "//a[contains(text(), '관람평')]").click()
        time.sleep(2)
    except:
        pass
    
    driver.execute_script("window.scrollTo(0, 1000);")
    time.sleep(2)
    reviews = driver.find_elements(By.CSS_SELECTOR, '.area_text_box, .desc_txt, li ._text')
    print(f" 리뷰 {len(reviews)}개 발견.")

    print(f"\n[Step 3] 데이터 병합 및 JSON 생성 중...")
    
    for review in reviews:
        try:
            text = review.text.strip().replace("관람객", "").replace("스포일러가 포함된 감상평입니다.", "").strip()
            if not text: continue
            
            try:
                score_text = review.find_element(By.XPATH, ".//ancestor::li//span[@class='num']").text
                score = int(score_text)
            except:
                score = 0

            label = "Sarcasm" if score <= 4 else "Non-Sarcasm"

            
            final_dataset.append({
                "Context": conversational_context,
                "Response": text,                  
                "Sarcasm_Label": label,             
                "Star_Score": score             
            })
            
        except Exception as e:
            continue


    file_name = 'movie_reviews_.json'
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(final_dataset, f, ensure_ascii=False, indent=4)
        
finally:
    driver.quit()

🚀 크롤러를 실행합니다 (Scenario: Blog Summary -> Dialog Context)...

📡 [Step 1] 블로그 접속 및 정보 수집...
   ✅ 블로그 Raw Data 수집 완료 (200자)
   ✨ Context 변환 완료:
B: 야, 너 이번에 개봉한 영화 '리얼' 봤어? A: 어, 블로그 찾아보니까 스토리와 연출이 난해하다는 평가가 많더라고. 완전 혹평 일색이던데? B: 진짜? 그 정도야? 사람들 반응은 어떤데?

📡 [Step 2] 영화 리뷰 수집 시작...
   ✅ 리뷰 66개 발견.

📡 [Step 3] 데이터 병합 및 JSON 생성 중...
